In [1]:
%display latex
import sympy
Parallelism().set(nproc=8)

We are using SageMath version 9.2 or later.

In [2]:
version()

'SageMath version 9.4, Release Date: 2021-08-22'

In [3]:
load("non_unimod_dim_3.sage")

# The case of $G_c$, $1 < c$

We write
$$
c = 1 + c_1^2, \qquad \text{where } 0 < c_1
$$

In [4]:
c1 = var("c1")
assume(c1>0)
c = 1 + c1^2
# forget(); c = var("c"); assume(c>1)
G = non_unimod_dim_3('G_c', c)
L = G.frame_left_inv()
L0, L1, L2 = L
R = G.frame_right_inv()
R0, R1, R2 = R

<string>:98: DeprecationWarning: 
Importing RealLine from here is deprecated. If you need to use it, please import it directly from sage.manifolds.differentiable.examples.real_line
See https://trac.sagemath.org/31881 for details.


Checking that the structure coefficients coincide with the ones of $\mathfrak g_c$

In [5]:
L0.bracket(L1).display(L)

[L_0,L_1] = 0

In [6]:
L2.bracket(L0).display(L)

[L_2,L_0] = L_1

In [7]:
L2.bracket(L1).display(L)

[L_2,L_1] = (-c1^2 - 1) L_0 + 2 L_1

## The left invariant metric $g = g_{\mu, \nu}$

We introduce the left invariant metric `g`, which in the frame `L` is given by the matrix
$$
\begin{pmatrix}
1&1&0\\
1&\mu&0\\
0&0&\nu
\end{pmatrix}, \qquad 1 < \mu \le c \text{ and } 0 < \nu
$$


In [8]:
mu, nu = var('mu nu')
assume(1<mu,mu<=c)
assume(0<nu)
g = G.left_inv_metric([1,1,0,1,mu,0,0,0,nu],name=r'g_{\mu,\nu}')
g.comp(L)[:]

[ 1  1  0]
[ 1 mu  0]
[ 0  0 nu]

We verify that the right invariant vector fields are Killing fields.

In [9]:
for X in R:
    display(g.lie_derivative(X).display(L))

0

0

0

We compute the intersecion of the orthogonal Lie algebra $\mathfrak{so}(g_{\mu, \nu}) \simeq \mathfrak{so}(3)$ with the pseudo-orthogonal Lie algebra with respecto to the Ricci tensor, $\mathfrak{so}(\operatorname{Ric}) \simeq \mathfrak{so}(2, 1)$

In [10]:
g_mat = mat_SR(g.comp(L)[:])
Ric = g.ricci()
Ric_mat = mat_SR(Ric.comp(L)[:]).simplify_full()
g_mat, Ric_mat

(
[ 1  1  0]  [                                        1/2*(c1^4 - mu^2 - 2*mu + 3)/((mu - 1)*nu)                   1/2*(c1^4 - 2*c1^2 + 2*(c1^2 + 1)*mu - 3*mu^2 + 1)/((mu - 1)*nu)                                                                                  0]
[ 1 mu  0]  [                  1/2*(c1^4 - 2*c1^2 + 2*(c1^2 + 1)*mu - 3*mu^2 + 1)/((mu - 1)*nu) 1/2*(2*c1^4 + mu^3 - 4*c1^2 - (c1^4 - 4*c1^2 - 17)*mu - 12*mu^2 - 6)/((mu - 1)*nu)                                                                                  0]
[ 0  0 nu], [                                                                                 0                                                                                  0                         -1/2*(c1^4 + 2*c1^2 - 2*(c1^2 - 1)*mu + mu^2 - 3)/(mu - 1)]
)

In [11]:
A_list = []
for i in range(3):
    for j in range(3):
        A_list += [var('a'+str(i)+str(j))]
A = matrix(3,3,A_list); A

[a00 a01 a02]
[a10 a11 a12]
[a20 a21 a22]

Equations for $A \in \mathfrak{so}(g_{\mu, \nu})$:

In [12]:
sols_so3 = solve((A.transpose() * g_mat + g_mat * A).list(), A.list(), algorithm='sympy')
A_so3 = A.subs(sols_so3); A_so3

[                                 -a11                               -a11*mu -a20*mu*nu/(mu - 1) + a21*nu/(mu - 1)]
[                                  a11                                   a11     a20*nu/(mu - 1) - a21*nu/(mu - 1)]
[                                  a20                                   a21                                     0]

The equations for `A_{so3}` being an element of $\mathfrak{so}(\operatorname{Ric})$ are not as simple as in the other cases. In fact, there are non-trivial solutions for certain values of $\mu$. 

In [13]:
(A_so3.transpose() * Ric_mat + Ric_mat * A_so3).factor()

[                                                2*(c1^2 - mu + 1)*a11/nu                                  -(c1^2 + mu - 3)*(c1^2 - mu + 1)*a11/nu                         -(a20*c1^2 - a20 + a21)*(c1^2 - mu + 1)/(mu - 1)]
[                                 -(c1^2 + mu - 3)*(c1^2 - mu + 1)*a11/nu                              -2*(c1^2 + 2*mu - 3)*(c1^2 - mu + 1)*a11/nu -(a20*c1^2 + 2*a20*mu - a21*mu - 3*a20 + 2*a21)*(c1^2 - mu + 1)/(mu - 1)]
[                        -(a20*c1^2 - a20 + a21)*(c1^2 - mu + 1)/(mu - 1) -(a20*c1^2 + 2*a20*mu - a21*mu - 3*a20 + 2*a21)*(c1^2 - mu + 1)/(mu - 1)                                                                        0]

We have to consider two posibilities for the above matrix equals zero.

## Case $\mu \neq c$

In this case $c_1^2 - \mu + 1 \neq 0$ and we can ask Sage to solve the equations for us in the usual way

In [14]:
sols_so3_Ric = solve((A_so3.transpose() * Ric_mat + Ric_mat * A_so3).list(), [a11,a20,a21], algorithm='sympy')
A_so3_Ric = A_so3.subs(sols_so3_Ric); A_so3_Ric

[0 0 0]
[0 0 0]
[0 0 0]

Therefore, the Lie algebra of the isotropy group of the full isometry group is trivial.

## The index of symmetry of $g_{\mu, \nu}$, $\mu \neq c$

Since the isometry group has dimension $3$, if the index of symmery is non-trivial, we will find a left invariant vector field $X$ such that $X_e \neq 0$ and $(\nabla X)_e = 0$

In [15]:
a0, a1, a2 = var('a0 a1 a2')
R_gen = G.right_inv_vect_field(a0,a1,a2,name=r'R_{gen}')
R_gen.display(R)

R_{gen} = a0 R_0 + a1 R_1 + a2 R_2

Define the Levi-Civita connection

In [16]:
nabla = g.connection()

And take the derivative of a generic right invariant vector field.

In [17]:
nablaR_gen = nabla(R_gen).at(G.zero)
ind_sym_eqs = nablaR_gen.comp()[:].factor()
ind_sym_eqs

[                                                       -1/2*(c1^2 + mu - 1)*a2/(mu - 1)                                                     -1/2*(c1^2 + mu - 1)*a2*mu/(mu - 1) 1/2*(a1*c1^2*mu - a0*c1^2 - 2*a1*c1^2 - a1*mu^2 - a0*mu + 3*a1*mu + a0 - 2*a1)/(mu - 1)]
[                                                        1/2*(c1^2 + mu - 1)*a2/(mu - 1)                                                         1/2*(c1^2 + mu - 1)*a2/(mu - 1)                          1/2*(a0*c1^2 + a1*c1^2 - a0*mu - 3*a1*mu + a0 + 3*a1)/(mu - 1)]
[                                                  -1/2*(a1*c1^2 - a1*mu - 2*a0 - a1)/nu                             -1/2*(a0*c1^2 + 2*a1*c1^2 - a0*mu - 4*a1*mu - a0 + 2*a1)/nu                                                                                       0]

Since $\mu > 1$, we conclude that $a_2 = 0$

In [18]:
ind_sym_eqs1 = ind_sym_eqs.subs(a2=0).factor()
ind_sym_eqs1

[                                                                                      0                                                                                       0 1/2*(a1*c1^2*mu - a0*c1^2 - 2*a1*c1^2 - a1*mu^2 - a0*mu + 3*a1*mu + a0 - 2*a1)/(mu - 1)]
[                                                                                      0                                                                                       0                          1/2*(a0*c1^2 + a1*c1^2 - a0*mu - 3*a1*mu + a0 + 3*a1)/(mu - 1)]
[                                                  -1/2*(a1*c1^2 - a1*mu - 2*a0 - a1)/nu                             -1/2*(a0*c1^2 + 2*a1*c1^2 - a0*mu - 4*a1*mu - a0 + 2*a1)/nu                                                                                       0]

We write $a_0$ in terms of $a_1$:

In [19]:
a0_func_a1 = ind_sym_eqs1[1,2].solve(a0)
a0_func_a1

[a0 == -(a1*c1^2 - 3*a1*mu + 3*a1)/(c1^2 - mu + 1)]

In [20]:
ind_sym_eqs1.subs(a0_func_a1).factor()

[                                                                         0                                                                          0       1/2*(c1^4 - 2*c1^2*mu + 2*c1^2 + mu^2 - 6*mu + 5)*a1/(c1^2 - mu + 1)]
[                                                                         0                                                                          0                                                                          0]
[-1/2*(c1^4 - 2*c1^2*mu + 2*c1^2 + mu^2 - 6*mu + 5)*a1/((c1^2 - mu + 1)*nu) -1/2*(c1^4 - 2*c1^2*mu + 2*c1^2 + mu^2 - 6*mu + 5)*a1/((c1^2 - mu + 1)*nu)                                                                          0]

So, the index of symmetry is non trivial if and only if
$$
c_{1}^{4} - 2 \, c_{1}^{2} \mu + 2 \, c_{1}^{2} + \mu^{2} - 6 \, \mu + 5 = 0
$$

In [21]:
mu_sol = solve(c1^4 - 2*c1^2*mu + 2*c1^2 + mu^2 - 6*mu + 5, mu)
mu_sol

[mu == c1^2 - 2*sqrt(c1^2 + 1) + 3]

Or, equivalently
$$
\mu = (\sqrt c - 1)^2 + 1
$$
Recall that such a $\mu$ satisfies $0 < \mu < c$. Therefore, the distribution of symmetry is spanned by the left invariant vector field
$$
\frac{c - 3\sqrt c + 2}{\sqrt{c} - 1} L_0 + L_1
$$

In [22]:
a0.subs(a0_func_a1).subs(a1=1).subs(mu_sol).factor()

(c1^2 - 3*sqrt(c1^2 + 1) + 3)/(sqrt(c1^2 + 1) - 1)


## Case $\mu = c$

Now we have to solve

In [23]:
(A_so3.transpose() * Ric_mat + Ric_mat * A_so3).subs(mu=c).factor()

[0 0 0]
[0 0 0]
[0 0 0]

This is a concequence of the metric being Einstein for $\mu = c$

In [24]:
(Ric_mat.subs(mu=c) + 2 / nu * g_mat.subs(mu=c)).factor()

[0 0 0]
[0 0 0]
[0 0 0]

Moreover, we can easily verify that the metric is symmetric when $\mu = c$ (and thus isometric to a hyperbolic space)

In [25]:
g1 = G.left_inv_metric([1,1,0,1,c,0,0,0,nu],name=r'g_{c,\nu}')
g1.comp(L)[:]

[       1        1        0]
[       1 c1^2 + 1        0]
[       0        0       nu]

In [26]:
nabla = g1.connection()
Riem = g1.riemann()
nablaRiem = nabla(Riem)
nablaRiem.display()

nabla_g_{c,\nu}(Riem(g_{c,\nu})) = 0

We can compute the constant curvature from the Ricci scalar

In [27]:
g1.ricci_scalar().expr()

-6/nu